Q1. Install MLflow

In [6]:

! mlflow --version


mlflow, version 2.13.1


In [7]:
! cd 02-experiment-tracking
! mkdir datasets 
! curl -o ./datasets/green_tripdata_2023-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
! curl -o ./datasets/green_tripdata_2023-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
! curl -o ./datasets/green_tripdata_2023-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1393k  100 1393k    0     0  33.1M      0 --:--:-- --:--:-- --:--:-- 33.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1497k  100 1497k    0     0  3707k      0 --:--:-- --:--:-- --:--:-- 3707k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1690k  100 1690k    0     0  4507k      0 --:--:-- --:--:-- --:--:-- 4495k
